### augmentation 

In [ ]:
import xml.etree.ElementTree as ET

def read_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    xml_file = 'C:/Users/ADMIN/practice/aug_practice/images/'

    bounding_box_list = []

    file_name = root.find('filename').text
    for obj in root.iter('object'):

        object_label = obj.find("name").text
        for box in obj.findall("bndbox"):
            x_min = int(box.find("xmin").text)
            y_min = int(box.find("ymin").text)
            x_max = int(box.find("xmax").text)
            y_max = int(box.find("ymax").text)

        bounding_box = [object_label, x_min, y_min, x_max, y_max]
        bounding_box_list.append(bounding_box)

    return bounding_box_list, file_name
    
    



In [ ]:
from os import listdir
import cv2
import numpy as np

def read_train_dataset(dir):
    images = []
    annotations = []
    

    for file in listdir(dir):
        if 'jpg' in file.lower() or 'png' in file.lower():
            images.append(cv2.imread(dir + file, 1))
            annotation_file = file.replace(file.split('.')[-1], 'xml')
            bounding_box_list, file_name = read_annotation(dir + annotation_file)
            annotations.append((bounding_box_list, annotation_file, file_name))

    images = np.array(images)

    return images, annotations



In [ ]:
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import matplotlib.pyplot as plt



## 바운딩박스를 포함한 이미지 augmentation

In [ ]:
import imgaug as ia
from imgaug import augmenters as iaa
from pascal_voc_writer import Writer

ia.seed(1)

dir = 'C:/Users/ADMIN/practice/aug_practice/images/'
images, annotations = read_train_dataset(dir)


for idx in range(len(images)):
    image = images[idx]
    boxes = annotations[idx][0]

    ia_bounding_boxes = []
    for box in boxes:
        ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

    bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)
    
    
    seq = iaa.Sequential([
        
        iaa.Cutout(nb_iterations=3 )
        

    ])
    

    

    seq_det = seq.to_deterministic()

    image_aug = seq_det.augment_images([image])[0]
    bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]


    new_dir = 'C:/Users/ADMIN/practice/aug_practice/augmented/'

    new_image_file = new_dir + 'after_' + annotations[idx][2]
    cv2.imwrite(new_image_file, image_aug)

    h, w = np.shape(image_aug)[0:2]
    voc_writer = Writer(new_image_file, w, h)

    for i in range(len(bbs_aug.bounding_boxes)):
        bb_box = bbs_aug.bounding_boxes[i]
        voc_writer.addObject(boxes[i][0], int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

    voc_writer.save(new_dir + 'after_' + annotations[idx][1])

#     image_before = bbs.draw_on_image(image, thickness=5)
#     image_after = bbs_aug.draw_on_image(image_aug, thickness=5, color=[0, 0, 255])

#     cv2.imshow('image_before', cv2.resize(image_before, (380, 640)))
#     cv2.imshow('image_after', cv2.resize(image_after, (380, 640)))

#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

    
    
    
    
    
    
    

## 이미지라벨링 확인

In [ ]:
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import xml.etree.ElementTree as Et
from xml.etree.ElementTree import Element, ElementTree


dataset_path = 'C:/Users/ADMIN/practice/aug_practice/check/'

IMAGE_FOLDER = 'C:/Users/ADMIN/practice/aug_practice/check/image/'
ANNOTATIONS_FOLDER = 'C:/Users/ADMIN/practice/aug_practice/check/label'

ann_root, ann_dir, ann_files = next(os.walk(os.path.join(dataset_path, ANNOTATIONS_FOLDER)))
img_root, amg_dir, img_files = next(os.walk(os.path.join(dataset_path, IMAGE_FOLDER)))

for xml_file in ann_files:

    
    img_name = img_files[img_files.index(".".join([xml_file.split(".")[0], "jpg"]))]
    img_file = os.path.join(img_root, img_name)
    image = Image.open(img_file).convert("RGB")
    draw = ImageDraw.Draw(image)

    xml = open(os.path.join(ann_root, xml_file), "r")
    tree = Et.parse(xml)
    root = tree.getroot()

    size = root.find("size")

    width = size.find("width").text
    height = size.find("height").text
    channels = size.find("depth").text

    objects = root.findall("object")

    for _object in objects:
        name = _object.find("name").text
        bndbox = _object.find("bndbox")
        xmin = int(bndbox.find("xmin").text)
        ymin = int(bndbox.find("ymin").text)
        xmax = int(bndbox.find("xmax").text)
        ymax = int(bndbox.find("ymax").text)

        # Box를 그릴 때, 왼쪽 상단 점과, 오른쪽 하단 점의 좌표를 입력으로 주면 됩니다.
        draw.rectangle(((xmin, ymin), (xmax, ymax)), outline="red")
        draw.text((xmin, ymin), name)

    plt.figure(figsize=(25,20))
    plt.imshow(image)
    plt.show()
    plt.close()

## Split folders

In [ ]:
import splitfolders
splitfolders.ratio('C:/Users/ADMIN/practice/images/', output = 'C:/Users/ADMIN/practice/output', 
                   seed=77, ratio=(0.8, 0.2), group_prefix=2)

## voc to yolo

In [ ]:
import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join

dirs = ['augmented']
classes = ['person', 'forklift', 'face', 'upperbody']

def getImagesInDir(dir_path):
    image_list = []
    for filename in glob.glob(dir_path + '/*.jpg'):
        image_list.append(filename)

    return image_list

def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def convert_annotation(dir_path, output_path, image_path):
    basename = os.path.basename(image_path)
    basename_no_ext = os.path.splitext(basename)[0]

    in_file = open(dir_path + '/' + basename_no_ext + '.xml')
    out_file = open(output_path + basename_no_ext + '.txt', 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

cwd = getcwd()

for dir_path in dirs:
    full_dir_path = cwd + '/' + dir_path
    output_path = full_dir_path +'/yolo/'

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    image_paths = getImagesInDir(full_dir_path)
    list_file = open(full_dir_path + '.txt', 'w')

    for image_path in image_paths:
        list_file.write(image_path + '\n')
        convert_annotation(full_dir_path, output_path, image_path)
    list_file.close()

    print("Finished processing: " + dir_path)

Finished processing: augmented


In [ ]:
cd C:/Users/ADMIN/practice/aug_practice/


C:\Users\ADMIN\practice\aug_practice


In [ ]:
import os
print(os.getcwd())


C:\Users\ADMIN\practice\aug_practice
